<a href="https://colab.research.google.com/github/namoshi/colab/blob/master/CLIP_Zero_Shot_Image_Recognition_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from PIL import Image
import requests
from transformers import CLIPProcessor, CLIPModel
import torch.nn.functional as F # 類似度計算のためにF.softmaxを使用

# 1. 事前学習済みCLIPモデルとProcessorのロード
# 使用するモデル名 (例: openai/clip-vit-base-patch32)
model_name = "openai/clip-vit-base-patch32"
try:
    # モデルとProcessorをロード
    model = CLIPModel.from_pretrained(model_name)
    processor = CLIPProcessor.from_pretrained(model_name)
    print(f"モデルを正常にロードしました: {model_name}")
except Exception as e:
    print(f"モデルのロード中にエラーが発生しました: {e}")
    print("インターネット接続を確認するか、モデル名が正しいか確認してください。")
    exit()

# デバイスの設定 (GPUが利用可能ならGPUを使用)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"使用デバイス: {device}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

モデルを正常にロードしました: openai/clip-vit-base-patch32
使用デバイス: cuda


In [2]:
# 2. 分類したいクラスに対応するテキストプロンプトの準備
# 分類したいクラス名のリスト
class_names = ["cat", "dog", "bird", "car", "house", "tree"]

# 各クラス名を適切なテキストプロンプトに変換（CLIPはこのような形式で学習されています）
# 例: "a photo of a cat"
text_inputs = [f"a photo of a {class_name}" for class_name in class_names]
print(f"\nクラス名: {class_names}")
print(f"テキストプロンプト: {text_inputs}")



クラス名: ['cat', 'dog', 'bird', 'car', 'house', 'tree']
テキストプロンプト: ['a photo of a cat', 'a photo of a dog', 'a photo of a bird', 'a photo of a car', 'a photo of a house', 'a photo of a tree']


In [3]:

# 3. 推論したい画像のロードと前処理
# 例として、インターネット上の画像を使用します。必要に応じてローカルの画像パスに変更してください。
# 画像URLの例（猫の画像）
image_url = "http://images.cocodataset.org/val2017/000000039769.jpg"

try:
    # URLから画像をダウンロードし、PILで開く
    image = Image.open(requests.get(image_url, stream=True).raw).convert("RGB")
    print(f"\nURLから画像を正常にロードしました: {image_url}")
    # image.show() # 画像を表示したい場合はこの行のコメントを外してください
except Exception as e:
    print(f"URLからの画像のロード中にエラーが発生しました: {e}")
    # ローカルの画像ファイルを使用する場合の例:
    # try:
    #     image_path = "path/to/your/image.jpg" # ここをあなたの画像ファイルパスに変更
    #     image = Image.open(image_path).convert("RGB")
    #     print(f"\nローカル画像を正常にロードしました: {image_path}")
    # except Exception as e_local:
    #     print(f"ローカル画像のロード中にエラーが発生しました: {e_local}")
    print("画像をロードできませんでした。終了します。")
    exit()

# 画像とテキストをProcessorで前処理し、モデル入力形式のテンソルに変換
# return_tensors="pt" でPyTorchテンソルとして取得
inputs = processor(text=text_inputs, images=image, return_tensors="pt", padding=True)

# モデルの入力データを適切なデバイスに移動
inputs = {k: v.to(device) for k, v in inputs.items()}



URLから画像を正常にロードしました: http://images.cocodataset.org/val2017/000000039769.jpg


In [4]:
# 4. CLIPモデルを使った画像特徴量とテキスト特徴量の計算
# 推論モードでは勾配計算は不要なので、torch.no_grad()を使用
with torch.no_grad():
    outputs = model(**inputs)

# モデルの出力から画像とテキストの特徴量を取得
# CLIPModelの出力には正規化された特徴量が含まれています
image_features = outputs.image_embeds
text_features = outputs.text_embeds


In [5]:
# 5. 画像特徴量とテキスト特徴量の類似度計算
# コサイン類似度を計算します。正規化された特徴量の場合、ドット積で計算できます。
# CLIPModelの出力には logit_scale が含まれており、これを使って類似度をスケーリングするのが一般的です。
logit_scale = model.logit_scale.exp()
logits_per_image = (image_features @ text_features.T) * logit_scale

# 類似度（logits）を確率に変換 (softmax)
probs = F.softmax(logits_per_image, dim=-1)

In [6]:
# 6. 類似度に基づいて最も高いクラスを予測として出力
# 最も確率の高いクラスのインデックスを取得
best_prob, best_idx = probs.max(dim=-1)

# 予測されたクラス名を取得
predicted_class = class_names[best_idx.item()]
confidence = best_prob.item()

print(f"\n予測結果:")
# 各クラスの確率を表示
for class_name, prob in zip(class_names, probs[0]):
    print(f"  {class_name}: {prob.item():.4f}")

print(f"\n予測されたクラス: {predicted_class}")
print(f"信頼度: {confidence:.4f}")


予測結果:
  cat: 0.9909
  dog: 0.0051
  bird: 0.0008
  car: 0.0022
  house: 0.0007
  tree: 0.0002

予測されたクラス: cat
信頼度: 0.9909
